In [1]:
import os
os.chdir("../")

In [2]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    params_image_size: list
    params_batch_size: int

In [7]:
from deepClassifier.constants import *
from deepClassifier.utils import *

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/PetImages"),
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
        

In [8]:
import tensorflow as tf
from pathlib import Path

class Evaluation:
    def __init__(self, config:EvaluationConfig) -> None:
        self.config = config
        
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    def save_score(self):
        scores = {"loss": self.score, "accuracy": self.score[1]}
        save_json(path= Path("scores.json"), data=scores)

In [9]:
try:
    config = ConfigurationManager()
    validation_config = config.get_validation_config()
    evaluation = Evaluation(config=validation_config)
    evaluation.evaluation()
    evaluation.save_score()
except  Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 1400s 3s/step - loss: 3.7972 - accuracy: 0.7635
